In [13]:
%pip install llama_index

Note: you may need to restart the kernel to use updated packages.


In [14]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
import sys
from dotenv import load_dotenv

In [15]:
load_dotenv()

True

In [3]:
import os
 
# get current directory
path = os.getcwd()
print("Current Directory", path)
 
# prints parent directory
print(os.path.abspath(os.path.join(path, os.pardir)))

Current Directory c:\Users\HP\Downloads\Ampufitness\backend\RAG
c:\Users\HP\Downloads\Ampufitness\backend


In [4]:
OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

In [5]:
COLLECTION_NAME=os.environ['COLLECTION_NAME']
DB_NAME=os.environ['DB_NAME']

In [9]:
from mongodb_service import mongodb_setup
db , mongo_client = mongodb_setup(DB_NAME=DB_NAME, COLLECTION_NAME=COLLECTION_NAME)

Connection to MongoDB successful


In [10]:
vector_store=MongoDBAtlasVectorSearch(mongodb_client=mongo_client)

In [11]:
sentence_index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
%pip install llama_index

In [19]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)
postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

c:\Users\HP\Downloads\Ampufitness\venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--BAAI--bge-reranker-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)